In [ ]:
#Cite this code as follows
#Sanghamita Bhoumik, Sayantan Chatterjee, Ankur Sarkar, Abhishek Kumar, and Ferdin Joe John Joseph. 2020. 
#Covid 19 Prediction from X Ray Images Using Fully Connected Convolutional Neural Network. 
#In CSBio '20: Proceedings of the Eleventh International Conference on Computational Systems-Biology and Bioinformatics (CSBio2020). 
#Association for Computing Machinery, New York, NY, USA, 106–107. DOI:https://doi.org/10.1145/3429210.3429233

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
print(os.listdir())

# Any results you write to the current directory are saved as output.

In [ ]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Conv2D,SeparableConv2D,add,GlobalAveragePooling2D
from keras.layers import MaxPooling2D,Concatenate
from keras.layers import Flatten,Reshape
from keras.layers import Dense,Input
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
# Initialising the CNN
classifier = Sequential()

In [ ]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(96, (3, 3), activation = 'relu'))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 2, activation = 'softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('./dataset',
                                                 target_size = (224,224),
                                                 batch_size = 50,
                                                 class_mode = 'categorical')

In [ ]:
import sys
from PIL import Image
sys.modules['Image'] = Image

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
#earlystopping_callback = EarlyStopping(monitor='accuracy')
checkpoint = ModelCheckpoint('output/{accuracy:.4f}.hdf5',monitor='accuracy', save_best_only=True, mode='auto')
import time
start_time = time.time()
classifier.fit_generator(training_set,epochs =10)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Part 3 - Making new predictions
import numpy as np
from keras.preprocessing import image
import imageio

#[os.path.join(item_dir, file) for file in all_item_dirs][:num_img]
item_dir="./DLAI3_CXR_Validation_Set"

l=list()
for items in os.listdir(item_dir):
    path_link=os.path.join(item_dir,items)
    test_image = image.load_img(path_link, target_size = (224,224))
    test_image = image.img_to_array(test_image)
    
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    training_set.class_indices
    if result[0][0] == 1:
        prediction = '1'
    else :
        prediction = '0'
    l.append({'filename':items,'label':prediction})
print('Prediction = ',l)

In [ ]:
sampleSubmission=pd.DataFrame(l,columns=["filename","label"])
sampleSubmission.to_csv('sampleSubmission1000epochs.csv',index=False)

sampleSubmission.head()